In [9]:
#!/usr/bin/env python3
"""
Preprocessing pipeline for Destatis population data (12411-0010)

- Parses raw Destatis CSV export
- Cleans metadata and quality flags
- Creates sliding-window regression dataset
- Performs (dummy) outlier detection
- Normalizes features
- Writes required output CSV files
"""

'\nPreprocessing pipeline for Destatis population data (12411-0010)\n\n- Parses raw Destatis CSV export\n- Cleans metadata and quality flags\n- Creates sliding-window regression dataset\n- Performs (dummy) outlier detection\n- Normalizes features\n- Writes required output CSV files\n'

In [10]:
import csv
import pathlib
import random
from typing import List, Tuple

import numpy as np
import pandas as pd

In [11]:
RAW_DATA_PATH = pathlib.Path("../data/raw/population_raw.csv")
OUTPUT_DIR = pathlib.Path("../data/preprocessed")

WINDOW_SIZE = 3        # sliding window length
TRAIN_SPLIT = 0.8
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [12]:
# =========================
# Utility functions
# =========================

def discard_outliers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Dummy outlier detection safeguard.

    Dataset is assumed to be outlier-free, so no rows are removed.
    """
    return df


def normalize_features(
    df: pd.DataFrame,
    feature_cols: List[str]
) -> Tuple[pd.DataFrame, dict]:
    """
    Z-score normalization.
    Returns normalized dataframe and normalization parameters.
    """
    stats = {}
    df_norm = df.copy()

    for col in feature_cols:
        mean = df[col].mean()
        std = df[col].std()

        if std == 0:
            std = 1.0

        df_norm[col] = (df[col] - mean) / std
        stats[col] = {"mean": mean, "std": std}

    return df_norm, stats

In [13]:
def load_destatis_csv(path: pathlib.Path) -> pd.DataFrame:
    """
    Parses the Destatis semicolon CSV with interleaved quality flags.
    Returns a wide dataframe: bundesland × year columns.
    """
    rows = []

    with path.open(encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=";")
        for row in reader:
            rows.append(row)

    # Find header row with years
    header_idx = next(
        i for i, r in enumerate(rows)
        if r and r[0] == "" and "31.12." in r[1]
    )

    year_row = rows[header_idx]
    years = []
    for cell in year_row:
        if cell.startswith("31.12."):
            years.append(int(cell[-4:]))

    data_rows = []
    for r in rows[header_idx + 1:]:
        if not r or r[0].startswith("_"):
            break
        if r[0].strip() == "":
            continue

        bundesland = r[0].strip()
        values = []

        # values are at positions 1,3,5,... (numbers), with "e" in between
        for i in range(1, len(r), 2):
            try:
                values.append(int(r[i]))
            except (ValueError, IndexError):
                break

        if len(values) == len(years):
            data_rows.append([bundesland] + values)

    columns = ["bundesland"] + years
    return pd.DataFrame(data_rows, columns=columns)

In [14]:
# =========================
# Feature engineering
# =========================

def build_sliding_window_dataset(
    df_wide: pd.DataFrame,
    window_size: int
) -> pd.DataFrame:
    """
    Converts wide bundesland×year data into supervised learning dataset.
    """
    records = []

    years = df_wide.columns[1:]

    for _, row in df_wide.iterrows():
        bundesland = row["bundesland"]
        series = row[1:].values.astype(float)

        for t in range(window_size, len(series)):
            features = series[t - window_size:t]
            target = series[t]
            year = years[t]

            record = {
                "bundesland": bundesland,
                "year": year,
                "target": target,
            }

            for i in range(window_size):
                record[f"x_t-{window_size-i}"] = features[i]

            records.append(record)

    return pd.DataFrame(records)

In [ ]:

# =========================
# Main pipeline
# =========================

def main():
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    df_wide = load_destatis_csv(RAW_DATA_PATH)

    df = build_sliding_window_dataset(df_wide, WINDOW_SIZE)

    df = discard_outliers(df)

    feature_cols = [c for c in df.columns if c.startswith("x_t-")]
    df_norm, _ = normalize_features(df, feature_cols)

    df_norm = df_norm.sample(frac=1.0, random_state=RANDOM_SEED).reset_index(drop=True)

    split_idx = int(len(df_norm) * TRAIN_SPLIT)
    train_df = df_norm.iloc[:split_idx]
    test_df = df_norm.iloc[split_idx:]

    activation_df = test_df.sample(n=1, random_state=RANDOM_SEED)

    df_norm.to_csv(OUTPUT_DIR / "joint_data_collection.csv", index=False)
    train_df.to_csv(OUTPUT_DIR / "training_data.csv", index=False)
    test_df.to_csv(OUTPUT_DIR / "test_data.csv", index=False)
    activation_df.to_csv(OUTPUT_DIR / "activation_data.csv", index=False)

    print("Preprocessing completed successfully.")
    print(f"Rows total: {len(df_norm)}")
    print(f"Training rows: {len(train_df)}")
    print(f"Test rows: {len(test_df)}")


if __name__ == "__main__":
    main()


Preprocessing completed successfully.
Rows total: 512
Training rows: 409
Test rows: 103


In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df_wide = load_destatis_csv(RAW_DATA_PATH)

df = build_sliding_window_dataset(df_wide, WINDOW_SIZE)

df = discard_outliers(df)

feature_cols = [c for c in df.columns if c.startswith("x_t-")]
df_norm, _ = normalize_features(df, feature_cols)

df_norm = df_norm.sample(frac=1.0, random_state=RANDOM_SEED).reset_index(drop=True)

split_idx = int(len(df_norm) * TRAIN_SPLIT)
train_df = df_norm.iloc[:split_idx]
test_df = df_norm.iloc[split_idx:]

In [ ]:

print("Preprocessing completed successfully.")
print(f"Rows total: {len(df_norm)}")
print(f"Training rows: {len(train_df)}")
print(f"Test rows: {len(test_df)}")

# VERIFY RESULTS

In [55]:
joint = pd.read_csv(OUTPUT_DIR / "joint_data_collection.csv")
train = pd.read_csv(OUTPUT_DIR / "training_data.csv")
test = pd.read_csv(OUTPUT_DIR / "test_data.csv")
activ = pd.read_csv(OUTPUT_DIR / "activation_data.csv")
joint.shape, train.shape, test.shape, activ.shape,

((512, 7), (400, 7), (112, 7), (1, 7))

In [56]:
activ

,bundesland,year,target,x_t-3,x_t-2,x_t-1,year-unnormalized
0,Hamburg,0.378701,1734272.0,-0.719083,-0.715811,-0.72986,2012


In [ ]:
test.groupby("bundesland").apply(pd.DataFrame)#.reset_index(level=1)
# d = pd.concat(test[])

/tmp/ipykernel_53099/1961452268.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test.groupby("bundesland").apply(pd.DataFrame)#.reset_index(level=1)


bundesland  year      target     x_t-3     x_t-2  \
bundesland                                                                      
Baden-Württemberg 21  Baden-Württemberg  1994  10272069.0  1.049005  1.076060   
                  40  Baden-Württemberg  2006  10738753.0  1.197437  1.197923   
                  69  Baden-Württemberg  2014  10716644.0  1.158731  1.166144   
                  95  Baden-Württemberg  2013  10631278.0  1.210615  1.154001   
Bayern            12             Bayern  2004  12443893.0  1.549257  1.555758   
...                                 ...   ...         ...       ...       ...   
Thüringen         35          Thüringen  1998   2462836.0 -0.562303 -0.564813   
                  47          Thüringen  2017   2151205.0 -0.636878 -0.633470   
                  50          Thüringen  2005   2334575.0 -0.586317 -0.590090   
                  60          Thüringen  2021   2108863.0 -0.639803 -0.641470   
                  94          Thüringen  2024   2100277.0 -0.647170 -0.644588   

                         x_t-1  
bundesland                      
Baden-Württemberg 21  1.090159  
                  40  1.197378  
                  69  1.175060  
                  95  1.161774  
Bayern            12  1.558073  
...                        ...  
Thüringen         35 -0.567440  
                  47 -0.635835  
                  50 -0.593700  
                  60 -0.643934  
                  94 -0.645081  

[103 rows x 6 columns]

In [57]:
joint.bundesland.value_counts()

bundesland
Baden-Württemberg         32
Bayern                    32
Berlin                    32
Brandenburg               32
Bremen                    32
Hamburg                   32
Hessen                    32
Mecklenburg-Vorpommern    32
Niedersachsen             32
Nordrhein-Westfalen       32
Rheinland-Pfalz           32
Saarland                  32
Sachsen                   32
Sachsen-Anhalt            32
Schleswig-Holstein        32
Thüringen                 32
Name: count, dtype: int64

In [58]:
train.bundesland.value_counts()

bundesland
Baden-Württemberg         25
Bayern                    25
Berlin                    25
Brandenburg               25
Bremen                    25
Hamburg                   25
Hessen                    25
Mecklenburg-Vorpommern    25
Niedersachsen             25
Nordrhein-Westfalen       25
Rheinland-Pfalz           25
Saarland                  25
Sachsen                   25
Sachsen-Anhalt            25
Schleswig-Holstein        25
Thüringen                 25
Name: count, dtype: int64

In [59]:
train.columns

Index(['bundesland', 'year', 'target', 'x_t-3', 'x_t-2', 'x_t-1',
       'year-unnormalized'],
      dtype='object')

In [60]:
test.bundesland.value_counts()

bundesland
Baden-Württemberg         7
Bayern                    7
Berlin                    7
Brandenburg               7
Bremen                    7
Hamburg                   7
Hessen                    7
Mecklenburg-Vorpommern    7
Niedersachsen             7
Nordrhein-Westfalen       7
Rheinland-Pfalz           7
Saarland                  7
Sachsen                   7
Sachsen-Anhalt            7
Schleswig-Holstein        7
Thüringen                 7
Name: count, dtype: int64

### -> preprocess.py could be improved by stratifying test/train datasets

# OTHER BS

In [42]:
loss = ([45939964248064.0, 45939951665152.0, 45939955859456.0, 45939947470848.0, 45939913916416.0, 45939884556288.0, 45939859390464.0, 45939800670208.0, 45939733561344.0, 45939637092352.0])
len(loss)

10